### Regression implemented only using Numpy

In [9]:
# import neccessary libraries
import numpy as np
import pandas as pd

In [11]:
# Load data and reshape for numpy execution
df = pd.read_excel("ArlHome5.xlsx")
df.columns = [c.strip().lower() for c in df.columns]

y = df["price"].to_numpy().reshape(-1, 1)
sqft = df["sqft"].to_numpy().reshape(-1, 1)
beds = df["beds"].to_numpy().reshape(-1, 1)
baths = df["baths"].to_numpy().reshape(-1, 1)

In [13]:
# Create the add_const function to calculate the constant
def add_const(*cols):
    """Stack a constant column with the provided columns."""
    X = np.hstack(cols)
    const = np.ones((X.shape[0], 1))
    return np.hstack([const, X])

In [17]:
# Create the ols_np function to execute regression
def ols_np(X, y):
    """Return dict with OLS results computed via NumPy only."""
    n, p1 = X.shape          # p1 includes the intercept
    k = p1 - 1               # number of predictors (excl. intercept)
    # Beta = (X'X)^(-1) X' y
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    beta = XtX_inv @ X.T @ y          # (p1 x 1)
    yhat = X @ beta                    # (n x 1)
    resid = y - yhat
    sse = float((resid.T @ resid))     # sum of squared errors
    sst = float(((y - y.mean()).T @ (y - y.mean())))
    mse = sse / (n - k - 1)            # residual mean squared error
    ser = np.sqrt(mse)                 # Standard Error of the Regression (Root MSE)
    r2 = 1 - sse/sst
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)

In [19]:
# Set up regression
def pretty_eq(beta, names):
    """beta: (p x 1), names includes 'const' then predictors"""
    b = beta.flatten()
    terms = [f"{b[0]:,.2f}"]
    for coef, nm in zip(b[1:], names[1:]):
        terms.append(f"{coef:,.2f}·{nm}")
    return " + ".join(terms)

In [23]:
# Display model results
print("Model 1 (const, sqft):  Price =", pretty_eq(m1["beta"], ["const","sqft"]))
print("Model 2 (const, sqft, beds):  Price =", pretty_eq(m2["beta"], ["const","sqft","beds"]))
print("Model 3 (const, sqft, baths): Price =", pretty_eq(m3["beta"], ["const","sqft","baths"]))
print("Model 4 (const, sqft, beds, baths): Price =", pretty_eq(m4["beta"], ["const","sqft","beds","baths"]))

Model 1 (const, sqft):  Price = 143,819.06 + 198.49·sqft
Model 2 (const, sqft, beds):  Price = 116,241.29 + 183.77·sqft + 17,632.92·beds
Model 3 (const, sqft, baths): Price = 132,770.88 + 118.30·sqft + 82,899.16·baths
Model 4 (const, sqft, beds, baths): Price = 111,454.76 + 107.67·sqft + 13,699.54·beds + 82,074.78·baths


In [25]:
# Metrics
summary = pd.DataFrame({
    "Model":   ["Model 1","Model 2","Model 3","Model 4"],
    "SER":     [m1["SER"], m2["SER"], m3["SER"], m4["SER"]],
    "Adj_R2":  [m1["adj_R2"], m2["adj_R2"], m3["adj_R2"], m4["adj_R2"]],
})
print("\nStandard Error (SER) and Adjusted R^2")
print(summary.to_string(index=False))


Standard Error (SER) and Adjusted R^2
  Model          SER   Adj_R2
Model 1 89842.116759 0.704630
Model 2 90562.377921 0.699875
Model 3 79359.431617 0.769536
Model 4 80147.139288 0.764938


In [27]:
# Predict a 2000 sq ft, 3 bed, 2 bath house with Models 1 and 4
new = np.array([[1, 2000],                  # for Model 1 (const, sqft)
               ])
pred_m1 = float(new @ m1["beta"])

new4 = np.array([[1, 2000, 3, 2]])          # for Model 4 (const, sqft, beds, baths)
pred_m4 = float(new4 @ m4["beta"])

print(f"\nPrediction (Model 1): ${pred_m1:,.2f}")
print(f"Prediction (Model 4): ${pred_m4:,.2f}")


Prediction (Model 1): $540,795.32
Prediction (Model 4): $532,035.81


C:\Users\aparnamehta\AppData\Local\Temp\ipykernel_30080\3246764926.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_m1 = float(new @ m1["beta"])
C:\Users\aparnamehta\AppData\Local\Temp\ipykernel_30080\3246764926.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_m4 = float(new4 @ m4["beta"])
